In [ ]:

# Torch
import torch
import torch.optim as optim
from torcheval.metrics import *
import torch.nn.functional as F
from torch.utils.data import DataLoader


# Benny pointnet
from pointnet2_benny import pointnet2_cls_ssg
import shutil

# Custom modules
from preprocessing_pre_fastsufer.preprocess import *
from preprocessing_post_fastsurfer.subject import *
from preprocessing_post_fastsurfer.vis import *
from ozzy_torch_utils.split_dataset import *
from ozzy_torch_utils.subject_dataset import *
from ozzy_torch_utils.plot import *
from ozzy_torch_utils.train_nn import *
from ozzy_torch_utils.model_parameters import *
from ozzy_torch_utils.init_dataloaders import *
from ensemble import *

from explain_pointnet import get_prediction


In [ ]:
dataset = SubjectDataset("/uolstore/home/student_lnxhome01/sc22olj/Compsci/year3/individual-project-COMP3931/individual-project-sc22olj/scratch-disk/full-datasets/hcampus-1.5T-cohort-holdout", ['CN', 'MCI'], load_in_memory = True, data_string='lhcampus_pointcloud_aligned', labels_string='research_group')

In [ ]:
dataloader = DataLoader()